# CYB80001 System Security Project
Prepared by **Derui (Derek) Wang**

---

# Session 3A - Deep Feedforward Neural Networks with Keras

**The purpose of this session is to demonstrate how to use TensorFlow's implementation of the Keras api to develop machine learning algorithms and deep neural network models. **



# Part 1. Classification with Keras

Let's repeat Session 2D with Keras and build a simple softmax classification with MNIST dataset.

* #### <span style="color:#0b486b">Step 1: Load or download the dataset</span>

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("datasets/")

W1011 23:37:46.087104 12848 deprecation.py:323] From <ipython-input-5-c72d886d7488>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1011 23:37:46.087104 12848 deprecation.py:323] From c:\users\windows\appdata\local\programs\python\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1011 23:37:46.110110 12848 deprecation.py:323] From c:\users\windows\appdata\local\programs\python\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting datasets/train-images-idx3-ubyte.gz


W1011 23:37:52.969605 12848 deprecation.py:323] From c:\users\windows\appdata\local\programs\python\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting datasets/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting datasets/t10k-images-idx3-ubyte.gz


W1011 23:37:54.059501 12848 deprecation.py:323] From c:\users\windows\appdata\local\programs\python\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting datasets/t10k-labels-idx1-ubyte.gz


* #### <span style="color:#0b486b">Step 2: Build a sequential model</span>


In Week 6, you saw the following TensorFlow code to construct a graph.

```
import tensorflow as tf
tf.reset_default_graph()

num_inputs = 28 * 28
num_hidden1 = 300
num_hidden2 = 100
num_outputs = 10
learning_rate = 0.01


x = tf.placeholder(tf.float32, shape=(None, num_inputs), name="x")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(x, num_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, num_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, num_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads = optimizer.compute_gradients(loss)    
    training_op = optimizer.apply_gradients(grads)    
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))   
    
init = tf.global_variables_initializer()
```

Below shows the Keras code to define such a model.

In [6]:
import tensorflow as tf
from tensorflow import keras

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01

model = keras.Sequential()

# Adds a densely-connected layer with n_hidden1 units to the model:
model.add(keras.layers.Dense(n_hidden1, activation='relu'))

# Add another densely-connected layer with n_hidden2 units :
model.add(keras.layers.Dense(n_hidden2, activation='relu'))

# Add a softmax layer with n_outputs output units:
model.add(keras.layers.Dense(n_outputs, activation='softmax'))


# Configure the learning process, including specifying the loss the evaluation metrics
model.compile(optimizer=tf.train.GradientDescentOptimizer(learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

W1011 23:37:54.375393 12848 deprecation.py:506] From c:\users\windows\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


* #### <span style="color:#0b486b">Step 3: Train and test the model</span>


Similarly the following code for training can be replaced.
``` 
num_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    print("Epoch\tTrain accuracy\tTest accuracy")
    for epoch in range(num_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
            
        acc_train = sess.run(accuracy,
                             feed_dict={x: x_batch, y: y_batch})
        
        acc_test = sess.run(accuracy,
                             feed_dict={x: mnist.test.images, y: mnist.test.labels})
        
        print("{}\t{}\t{}".format(epoch, acc_train))   
``` 

The training is simple with Keras.

In [7]:
num_epochs = 20
batch_size = 50

model.fit(mnist.train.images, 
          mnist.train.labels, 
          epochs=num_epochs, 
          batch_size=batch_size, 
          validation_data=( mnist.test.images, mnist.test.labels))

Train on 55000 samples, validate on 10000 samples
Epoch 1/20
55000/55000 [==============================] - 3s 52us/sample - loss: 0.7254 - sparse_categorical_accuracy: 0.8203 - val_loss: 0.3488 - val_sparse_categorical_accuracy: 0.9066
Epoch 2/20
55000/55000 [==============================] - 2s 40us/sample - loss: 0.3280 - sparse_categorical_accuracy: 0.9075 - val_loss: 0.2776 - val_sparse_categorical_accuracy: 0.9216
Epoch 3/20
55000/55000 [==============================] - 2s 40us/sample - loss: 0.2737 - sparse_categorical_accuracy: 0.9220 - val_loss: 0.2419 - val_sparse_categorical_accuracy: 0.9296
Epoch 4/20
55000/55000 [==============================] - 2s 39us/sample - loss: 0.2406 - sparse_categorical_accuracy: 0.9315 - val_loss: 0.2184 - val_sparse_categorical_accuracy: 0.9382
Epoch 5/20
55000/55000 [==============================] - 2s 42us/sample - loss: 0.2161 - sparse_categorical_accuracy: 0.9385 - val_loss: 0.2021 - val_sparse_categorical_accuracy: 0.9425
Epoch 6/20
5500

An added benefit of using Keras is that the `model` Python variable preserves all the information regarding the model, including the current weights. So we can easily further improve the model with additional training epochs.

In [8]:
history = model.fit(mnist.train.images, 
                    mnist.train.labels,
                    epochs=5,
                    batch_size=batch_size, 
                    validation_data=( mnist.test.images, mnist.test.labels))

Train on 55000 samples, validate on 10000 samples
Epoch 1/5
55000/55000 [==============================] - 2s 40us/sample - loss: 0.0773 - sparse_categorical_accuracy: 0.9789 - val_loss: 0.0965 - val_sparse_categorical_accuracy: 0.9719
Epoch 2/5
55000/55000 [==============================] - 2s 39us/sample - loss: 0.0738 - sparse_categorical_accuracy: 0.9802 - val_loss: 0.0925 - val_sparse_categorical_accuracy: 0.9737
Epoch 3/5
55000/55000 [==============================] - 2s 40us/sample - loss: 0.0707 - sparse_categorical_accuracy: 0.9809 - val_loss: 0.0910 - val_sparse_categorical_accuracy: 0.9727
Epoch 4/5
55000/55000 [==============================] - 2s 40us/sample - loss: 0.0674 - sparse_categorical_accuracy: 0.9816 - val_loss: 0.0881 - val_sparse_categorical_accuracy: 0.9732
Epoch 5/5
55000/55000 [==============================] - 2s 41us/sample - loss: 0.0645 - sparse_categorical_accuracy: 0.9825 - val_loss: 0.0900 - val_sparse_categorical_accuracy: 0.9735


### Excercises


- Change five different value for the number of hidden nodes in each layer and report the best numbers among your chosen number.
- Increase the number of hidden layers to **three** and set five values for the number of hidden nodes then report the best value and its performance.
- Try to change the optimizer to train the model to [Adam](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) and [RMSProp](https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer)

# Part 2. Saving and loading Keras models

* #### <span style="color:#0b486b">Step 1: Build a CNN using Keras Model API</span>

We first build a convolutional neural network using **Keras Model API**.

In [20]:
from keras.models import Model
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import Dense, Dropout, Activation, Flatten, Activation, Multiply, BatchNormalization
from keras.layers import LeakyReLU, PReLU
from keras.optimizers import SGD

* #### <span style="color:#0b486b">Step 1: Building model using the Model API</span>

We build the CNN model by using a function named `MNIST_CNN`. We do not use the sequential model here. Instead, we apply the layers from Keras `layers` module as functions and then use the `Model` API to wrap the layers into a model. 

The model is then compiled with a proper loss functions and an optimizer used for training. We also define the metric used for monitoring the performance on the validation data during training.

In [16]:
def MNIST_CNN(In_Shape):
    inputs = Input(shape=In_Shape, name='Normal_inputs')
    x = Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='C1')(inputs)
    x = Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='C2')(x)
    x = MaxPooling2D(pool_size=(2,2), name='MP1')(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='C3')(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='C4')(x)
    x = MaxPooling2D(pool_size=(2,2), name='MP2')(x)
    x = Flatten(name='Flatten')(x)
    x = Dense(200, activation='relu', name='D1')(x)
    x = Dense(200, activation='relu', name='D2')(x)
    x = Dense(10, name='logits')(x)
    outputs = Activation('softmax', name='normal_output')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    return model

* #### <span style="color:#0b486b">Step 2: Train the CNN</span>

The CNN is taking inputs in the sahpe of **Height x Weight x Channel (HWC)**. Hence, we first need to reshape each example to shape (28,28,1). In other words, we can reshape the entire dataset to **(-1,28,28,1)**. 

Next, we convert the labels into one-hot-vectors for training using non-sparese loss functions. WE can use the `to_categorical` function from `keras.utils` to achieve this.

In [24]:
from keras.utils import to_categorical

num_epochs = 20
batch_size = 50

mnist_train_images = mnist.train.images.reshape((-1,28,28,1))
mnist_train_labels = to_categorical(mnist.train.labels)
mnist_test_images = mnist.test.images.reshape((-1,28,28,1))
mnist_test_labels = to_categorical(mnist.test.labels)
print(mnist_train_images.shape)
print(mnist_train_labels.shape)

mnist_cnn = MNIST_CNN(mnist_train_images[0].shape)
mnist_cnn.fit(mnist_train_images, 
              mnist_train_labels,
              epochs=num_epochs,
              batch_size=batch_size, 
              validation_data=(mnist_test_images, mnist_test_labels))

(55000, 28, 28, 1)
(55000, 10)
Train on 55000 samples, validate on 10000 samples
Epoch 1/20
55000/55000 [==============================] - 9s 156us/step - loss: 0.1281 - acc: 0.9590 - val_loss: 0.0393 - val_acc: 0.9872
Epoch 2/20
55000/55000 [==============================] - 8s 146us/step - loss: 0.0409 - acc: 0.9872 - val_loss: 0.0312 - val_acc: 0.9900
Epoch 3/20
55000/55000 [==============================] - 8s 146us/step - loss: 0.0288 - acc: 0.9910 - val_loss: 0.0353 - val_acc: 0.9898
Epoch 4/20
55000/55000 [==============================] - 8s 140us/step - loss: 0.0202 - acc: 0.9937 - val_loss: 0.0244 - val_acc: 0.9923
Epoch 5/20
55000/55000 [==============================] - 8s 150us/step - loss: 0.0176 - acc: 0.9945 - val_loss: 0.0291 - val_acc: 0.9906
Epoch 6/20
55000/55000 [==============================] - 8s 147us/step - loss: 0.0149 - acc: 0.9955 - val_loss: 0.0264 - val_acc: 0.9921
Epoch 7/20
55000/55000 [==============================] - 8s 147us/step - loss: 0.0125 - ac

* #### <span style="color:#0b486b">Step 3: Save the trained model</span>

We can simple call the `save` method on the trained model to save it. We recommend saving the model using `.h5` format.

In [26]:
mnist_cnn.save('mnist_cnn_model.h5')

* #### <span style="color:#0b486b">Step 4: Load pre-trained model</span>

To load a pre-trained Keras model, we can use the `load_model` API.

In [29]:
from keras.models import load_model

loaded_model = load_model('mnist_cnn_model.h5')
print('Evaluate the loaded model')
print(f'Accuracy of the loaded model: {loaded_model.evaluate(mnist_test_images, mnist_test_labels)[1]}')

Evaluate the loaded model
10000/10000 [==============================] - 1s 82us/step
Accuracy of the loaded model: 0.991


---
### <div  style="text-align:center">**THE END**</div>